In [2]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [3]:
# Load in data, i will need to build a new dataframe, combining features of all them 
df_properties = pd.read_csv(r"C:\Users\mrice\OneDrive\Documents\Data Science - Callum\Projects\Financial Projects\london_house_prices\data_gathering\london_flats.csv")
print(df_properties.columns)

df_stations = pd.read_csv(r"C:\Users\mrice\OneDrive\Documents\Data Science - Callum\Projects\Financial Projects\london_house_prices\data_gathering\stations.csv")
print(df_stations.columns)

df_crime = pd.read_csv(r"C:\Users\mrice\OneDrive\Documents\Data Science - Callum\Projects\Financial Projects\london_house_prices\data_gathering\crime_data.csv")
print(df_crime.columns)

df_amenities = pd.read_csv(r"C:\Users\mrice\OneDrive\Documents\Data Science - Callum\Projects\Financial Projects\london_house_prices\data_gathering\amenities.csv")
print(df_amenities.columns)

Index(['address', 'price', 'description', 'bedrooms', 'bathrooms', 'latitude',
       'longitude', 'postcode', 'property_id'],
      dtype='object')
Index(['property_id', 'latitude', 'longitude', 'closest_station', 'distance'], dtype='object')
Index(['property_id', 'latitude', 'longitude', 'crime'], dtype='object')
Index(['property_id', 'latitude', 'longitude', 'park', 'gym', 'shop'], dtype='object')


In [4]:
# Drop duplicate columns
df_stations = df_stations.drop(columns=['latitude', 'longitude'], errors = 'ignore')

df_crime = df_crime.drop(columns=['latitude', 'longitude'], errors = 'ignore')

df_amenities = df_amenities.drop(columns=['latitude', 'longitude'], errors = 'ignore')

In [5]:
df_properties['property_id'] = df_properties['property_id'].str.replace("\r", "")

In [6]:
# Merge all dataframes
df = pd.merge(df_properties, df_stations, how = 'left', on = 'property_id')

df = pd.merge(df, df_crime, how = 'left', on = 'property_id')

df = pd.merge(df, df_amenities, how = 'left', on = 'property_id')

df = df.drop_duplicates()

print(df.columns)


Index(['address', 'price', 'description', 'bedrooms', 'bathrooms', 'latitude',
       'longitude', 'postcode', 'property_id', 'closest_station', 'distance',
       'crime', 'park', 'gym', 'shop'],
      dtype='object')


In [7]:
# Handling missing values

df['distance'].replace([float('inf'), -float('inf')], 2, inplace=True)

df['closest_station'] = df['closest_station'].fillna("Unknown")

df['distance'] = df['distance'].fillna(2)

print(df.shape)

(982, 15)


C:\Users\mrice\AppData\Local\Temp\ipykernel_22988\1246776736.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['distance'].replace([float('inf'), -float('inf')], 2, inplace=True)


In [8]:
print(df[df['distance'] == float('inf')])

Empty DataFrame
Columns: [address, price, description, bedrooms, bathrooms, latitude, longitude, postcode, property_id, closest_station, distance, crime, park, gym, shop]
Index: []


In [9]:
print(df['price'].dtype)

int64


In [10]:
# Make df suitable for user preference
df['gym_near'] = df['gym'].apply(lambda x: 0 if x == 'No gym' else 1)
df['park_near'] = df['park'].apply(lambda x: 0 if x == 'No park' else 1)
df['shop_near'] = df['shop'].apply(lambda x: 0 if x == 'No shop' else 1)

# Alter crime column
df['crime_rate'] = pd.qcut(df['crime'], q = 3, labels=['low', 'med', 'high'])

In [11]:
print(df.columns)

Index(['address', 'price', 'description', 'bedrooms', 'bathrooms', 'latitude',
       'longitude', 'postcode', 'property_id', 'closest_station', 'distance',
       'crime', 'park', 'gym', 'shop', 'gym_near', 'park_near', 'shop_near',
       'crime_rate'],
      dtype='object')


In [27]:
# Gather user preferences
def gather_user_input():
    min_rooms = int(input("Min rooms: "))
    max_rooms = int(input("Max rooms: "))
    min_price = int(input("Min price: "))
    max_price = int(input("Max_price: "))
    tube_distance = float(input("Max distance from tube station: "))

    return {
        'min_rooms': min_rooms,
        'max_rooms': max_rooms,
        'min_price': min_price,
        'max_price': max_price,
        'tube_distance': tube_distance
    }



In [28]:
# Conect to SQL database
import mysql.connector

db_conncetion = mysql.connector.connect(
    host = 'localhost',
    username = 'root',
    password = 'James@0405',
    database = 'london_properties'
)

cursor = db_conncetion.cursor()

In [33]:
# Build dynamic query for user preferences
def construct_sql_query(user_prefs):
    query = """
    SELECT p.*  
    FROM properties p 
    JOIN tubes t ON p.property_id = t.property_id
    WHERE 1=1
    """
    params = []

    if user_prefs['min_rooms']:
        query += " AND p.bedrooms >= %s"
        params.append(user_prefs['min_rooms'])
    
    if user_prefs['max_rooms']:
        query += " AND p.bedrooms <= %s"
        params.append(user_prefs['max_rooms'])

    if user_prefs['min_price']:
        query += " AND p.price >= %s"
        params.append(user_prefs['min_price'])

    if user_prefs['max_price']:
        query += " AND p.price <= %s"
        params.append(user_prefs['max_price'])
    
    if user_prefs['tube_distance']:
        query += " AND t.distance <= %s"
        params.append(user_prefs['tube_distance'])
    
    return query, params

    

In [34]:
# retreve fltered properties
def get_filtererd_properties(user_prefs):
    query, params = construct_sql_query(user_prefs)

    cursor.execute(query, params)
    results = cursor.fetchall()

    df = pd.DataFrame(results, columns=[column[0] for column in cursor.description])

    return df


In [35]:
user_prefs = gather_user_input()
filtered_properties = get_filtererd_properties(user_prefs)

if filtered_properties.empty:
    print("No Porperties match criteria")
else:
    print(filtered_properties)

                                       address  price  \
0              Wilberforce Road, Finsbury Park   2500   
1    Moluccas Point, 24 Canal Approach, London   2200   
2                                 Ashmore Road   2500   
3                    Coxwell Boulevard, London   2400   
4               Mazenod Avenue, West Hampstead   1842   
..                                         ...    ...   
127                  Berrymead Gardens, London   2200   
128                               Kings Parade   2250   
129                       Harvard Road, London   2000   
130                      Banister Road, London   2500   
131                        Aubert Park, London   2400   

                                           description  bedrooms  bathrooms  \
0    This rather beautiful first floor maisonette w...         2          1   
1    Key features Brand New DevelopmentFully Furnis...         2          0   
2    Bright and airy apartment located on the top f...         2          2   

In [ ]:
# Create tfidfvectorizer
tfidf = TfidfVectorizer(stop_words='english', max_features=15)
tfidf_matrix = tfidf.fit_transform(df_properties['description'])
print(tfidf_matrix.shape)

(982, 15)


In [62]:
# Calculate cosine similarity with itself
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Create indices variable
indices = pd.Series(df_properties.index, index=df_properties['address']).drop_duplicates()

In [63]:
def recommendation_system(address, cosine_sim=cosine_sim, num_recommended=5):
    idx = indices[address]  # Get index of the address
    sim_scores = list(enumerate(cosine_sim[idx].flatten()))  # Flatten in case it's a 2D array
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    top_sim = sim_scores[1:num_recommended+1]  # Skip the first one as it's the property itself
    property_indices = [i[0] for i in top_sim]
    
    return df_properties['address'].iloc[property_indices]




In [64]:
print(df_properties['address'])

0               Merchant Square, Paddington
1                      Grove Avenue, Sutton
2           Wilberforce Road, Finsbury Park
3               Strathmore Court, Park Road
4                      Graveney Road London
                       ...                 
977          Wandsworth Bridge Road, Fulham
978    One Park Drive, Canary Wharf, London
979      Kensington High Street, Kensington
980            Ansleigh Place, Notting Hill
981             Cleveland Square, Bayswater
Name: address, Length: 982, dtype: object


In [65]:
recommendation_system('Grove Avenue, Sutton', num_recommended=5)

744    Green Lane, Hendon, London
272         Bloxhall Road, Leyton
817        Ridgeway Views, London
47     Crossharbour Plaza, London
56      St. Pauls Terrace, London
Name: address, dtype: object

In [66]:
print(df_properties.iloc[817])

address                                   Ridgeway Views, London
price                                                       1850
description    Brand new one bedroom apartments in Mill Hill,...
bedrooms                                                       1
bathrooms                                                      1
latitude                                               51.617546
longitude                                              -0.219787
postcode                                                     NW7
property_id                     8d6192ce6bb753a0e043b4a5cc72c957
Name: 817, dtype: object


In [61]:
print(df_properties.iloc[1])

address                                     Grove Avenue, Sutton
price                                                       1675
description    **AVAILABLE APRIL**Leaders are pleased to offe...
bedrooms                                                       2
bathrooms                                                      1
latitude                                               51.359806
longitude                                               -0.20128
postcode                                                 SM1 2DA
property_id                     0eaca9c01a72e47b55656d34229fbbf1
Name: 1, dtype: object
